<a href="https://colab.research.google.com/github/amtayl25/1d3d/blob/main/AdvDiff_DG_Bif.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title Install required libraries (try-catch safe)
"""
NOTE: You can save the Docker image state after running this block so that you don't have to run it every time you start a new environment.
"""
import os, re

def replace_in_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()

    # Replace 'ufl' with 'ufl_legacy'
    content = re.sub(r'\bufl\b', 'ufl_legacy', content)

    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(content)

def process_directory(directory):
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith('.py'):
                file_path = os.path.join(root, file)
                replace_in_file(file_path)

# ipywidgets
try:
    import ipywidgets
except ImportError:
    !pip install ipywidgets

# dolfin
try:
    import dolfin
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/fenics-install-release-real.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
    import dolfin

# block
try:
    import block
except ImportError:
    !git clone "https://bitbucket.org/fenics-apps/cbc.block/src/master/"
    !pip install master/

# fenics_ii
try:
    import xii
except ImportError:
    !git clone "https://github.com/MiroK/fenics_ii"
    process_directory("fenics_ii/")
    !pip install fenics_ii/

# vtk
try:
    import vtk
except ImportError:
    !pip install vtk

# graphnics
try:
    import graphnics
except ImportError:
    !git clone "https://github.com/IngeborgGjerde/graphnics"
    !pip install graphnics/

# meshio
try:
    import meshio
except ImportError:
    !pip install meshio

# pyvista
try:
    import pyvista
except ImportError:
    !pip install pyvista

--2026-01-09 16:26:01--  https://fem-on-colab.github.io/releases/fenics-install-release-real.sh
Resolving fem-on-colab.github.io (fem-on-colab.github.io)... 185.199.109.153, 185.199.111.153, 185.199.110.153, ...
Connecting to fem-on-colab.github.io (fem-on-colab.github.io)|185.199.109.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4180 (4.1K) [application/x-sh]
Saving to: ‘/tmp/fenics-install.sh’

/tmp/fenics-install 100%[===================>]   4.08K  --.-KB/s    in 0s      

2026-01-09 16:26:01 (49.2 MB/s) - ‘/tmp/fenics-install.sh’ saved [4180/4180]

+ INSTALL_PREFIX=/usr/local
++ echo /usr/local
++ awk -F/ '{print NF-1}'
+ INSTALL_PREFIX_DEPTH=2
+ PROJECT_NAME=fem-on-colab
+ SHARE_PREFIX=/usr/local/share/fem-on-colab
+ FENICS_INSTALLED=/usr/local/share/fem-on-colab/fenics.installed
+ [[ ! -f /usr/local/share/fem-on-colab/fenics.installed ]]
+ PYBIND11_INSTALL_SCRIPT_PATH=https://github.com/fem-on-colab/fem-on-colab.github.io/raw/f34c0f89/releases/pybi

In [3]:
#@title Define functions
from math import gamma
from argparse import RawDescriptionHelpFormatter
from petsc4py import PETSc
from dolfin import *
from xii import *
import numpy as np
import time
import networkx as nx
from graphnics import FenicsGraph
from scipy.spatial import cKDTree
from itertools import combinations
from collections import deque

def bif_tangents(G):
    pos   = nx.get_node_attributes(G,"pos")
    nodes = list(G.nodes)
    T     = np.zeros((len(nodes),3))
    for k, n in enumerate(nodes):
        p0 = np.array(pos[n])
        neighbor = list(G.neighbors(n))
        vec = np.zeros(3)
        for m in neighbor:
            vec += np.array(pos[m]) - p0

        norm = np.linalg.norm(vec)
        if norm > 0:
          T[k,:] = vec/norm
        else:
          T[k,:] = np.array([0,0,1.0]) #use unit normal if needed
    P = np.array([pos[n] for n in nodes])
    return nodes, P, T

class TangentExpression(UserExpression):
    def __init__(self, nodes, P, T_array, **kwargs):
        self.nodes  = nodes
        self.P      = np.asarray(P)
        self.T_array = np.asarray(T_array)
        self.kdtree = cKDTree(self.P)
        super().__init__(**kwargs)
    def eval(self,values,x):
      _, idx = self.kdtree.query((x[0],x[1],x[2]))
      tx, ty, tz = self.T_array[idx]
      values[0] = tx
      values[1] = ty
      values[2] = tz
    def value_shape(self):
      return (3,)

def make_Y_bifurcation(
    parent_len=0.45, parent_pts=4,          # parent vessel
    daughter_len=0.45, daughter_pts=4,      # identical daughter vessels
    angle_deg=35.0,                         # opening half-angle of the Y (each daughter)
    r_parent=0.05, r_daughter=0.05/np.sqrt(2),        # radii
    damage_default=0.0,                     # damage field included in vtk file from Pierce's example, just made zero
    origin=(0.0, 0.0, -0.45),               # parent begins at z = -0.45
    bifurcation_z= 0.0                      # split occors at origin
):
    G = FenicsGraph()

    # === Parent: from (0,0,-0.45) to (0,0,bifurcation_z) ===
    z0 = origin[2]
    parent_zs = np.linspace(z0, bifurcation_z, parent_pts)
    parent_ids = []
    for z in parent_zs:
        i = len(G)  # next node id
        G.add_node(i, pos=(origin[0], origin[1], z), radius=r_parent, damage=damage_default)
        parent_ids.append(i)

    # Use the last parent node as the bifurcation node:
    bif_id = parent_ids[-1]
    bif_pt = G.nodes[bif_id]["pos"]

    # Connect parent and daughter paths
    for a, b in zip(parent_ids[:-1], parent_ids[1:]):
        G.add_edge(a, b)

    # === Daughters: symmetric about the z-axis in x–z plane ===
    angle = np.deg2rad(angle_deg)
    # Unit directions for the two daughters (right and left in +x/-x):
    d1 = np.array([ np.sin(angle), 0.0, np.cos(angle)])
    d2 = np.array([-np.sin(angle), 0.0, np.cos(angle)])

    def add_branch(bif_id, dir_vec):
        # start at the bifurcation point (re-use the node id at the bifurcation)
        # then create additional points along the ray
        ids = [bif_id]
        for k in range(1, daughter_pts):
            p = np.array(bif_pt) + (k / (daughter_pts - 1)) * daughter_len * dir_vec
            i = len(G)
            G.add_node(i, pos=tuple(p), radius=r_daughter, damage=damage_default)
            ids.append(i)
        # connect the line
        for a, b in zip(ids[:-1], ids[1:]):
            G.add_edge(a, b)

    add_branch(bif_id, d1)
    add_branch(bif_id, d2)

    return G

def build_bifurcation_pairs(G):
    """
    For every bifurcation node (degree > 2), return ordered pairs:
      (parent, daughter) for each daughter, and (daughter_i, daughter_j) for all daughter pairs.

    """
    pos = nx.get_node_attributes(G, 'pos')
    bif_nodes = [n for n in G.nodes if G.degree[n] > 2]
    results = []

    for b in bif_nodes:
        neighbor = list(G.neighbors(b))
        # Sort neighbors by z so smallest-z branch is treated as "parent" (upstream)
        neighbor_sorted = sorted(neighbor, key=lambda n: pos[n][2])
        parent_vessel   = neighbor_sorted[0]
        daughter_vessel = neighbor_sorted[1:]


        # Assign local vessel IDs per neighbor by walking to next junction/terminal
        vessel_id = {}
        for k, n in enumerate(neighbor_sorted, start = 1):
            vessel_id[n] = k

        parent_id = vessel_id[parent_vessel]
        daughter_id = [vessel_id[n] for n in daughter_vessel]

        results.append({
            "bif_node": b,
            "parent_id": parent_id,
            "daughter_id": daughter_id,
            "pairs": [(parent_id,d) for d in daughter_id] + list(combinations(daughter_id, 2)),
            "pos": np.array(pos[b]),
            "neighbor_sorted": neighbor_sorted,
            "vessel_id": vessel_id
        })

    return results

def bifurcation_vertices_from_graph(Lambda, bif_info):
    """Return mesh vertex indices of bifurcation points (nearest vertex to each bif node position)."""
    coords = Lambda.coordinates()

    def nearest_vertex(point):
        diff = coords - point
        return int(np.argmin(np.linalg.norm(diff, axis=1)))

    bif_vertices = []
    for bif in bif_info:
        vtx = nearest_vertex(bif["pos"])
        bif_vertices.append(vtx)
    return sorted(set(bif_vertices))


def add_kirchhoff_junction_terms_P1DG(
    A_petsc, V3, V1, Lambda, bif_vertices, D_area_expr,
    eta_gamma=10.0, epsilon=1.0
):
    """
    Assemble Kirchhoff/SIPG junction terms by writing directly into the PETSc Mat.

    Works even when dolfin.PETScMatrix has no .add() and is not subscriptable.
    """
    dm = V1.dofmap()
    coords = Lambda.coordinates()

    Lambda.init(0, 1)
    v2c = Lambda.topology()(0, 1)

    N = A_petsc.mat().getSize()[0]
    off1 = N - V1.dim()   # assumes V1 block is last in ii_convert ordering

    # get petsc4py Mat
    M = A_petsc.mat()

    M.setOption(PETSc.Mat.Option.NEW_NONZERO_ALLOCATION_ERR, False)
    M.setOption(PETSc.Mat.Option.NEW_NONZERO_LOCATION_ERR, False)

    def add_entry(I, J, val):
        # addv=True means accumulate into existing entry
        M.setValue(I, J, val, addv=True)

    for vtx in bif_vertices:
        incident_cells = list(v2c(vtx))
        m = len(incident_cells)
        if m < 3:
            continue

        branches = []
        h_gamma = 0.0

        for c in incident_cells:
            cell = Cell(Lambda, c)
            vids = cell.entities(0)
            x0 = coords[vids[0]]
            x1 = coords[vids[1]]
            hK = float(np.linalg.norm(x1 - x0))
            h_gamma = max(h_gamma, hK)

            if vids[0] == vtx:
                loc_g, loc_o = 0, 1
                x_mid = 0.5*(x0 + x1)
            else:
                loc_g, loc_o = 1, 0
                x_mid = 0.5*(x0 + x1)

            cdofs = dm.cell_dofs(c)  # DG1: 2 dofs per cell
            dof_g = int(cdofs[loc_g])
            dof_o = int(cdofs[loc_o])

            Da = float(D_area_expr(Point(*x_mid)))
            branches.append(dict(hK=hK, Da=Da, dof_g=dof_g, dof_o=dof_o))

        for a in range(m):
            for b in range(a + 1, m):
                Ia = branches[a]
                Ib = branches[b]

                ha, Da = Ia["hK"], Ia["Da"]
                hb, Db = Ib["hK"], Ib["Da"]
                ga, oa = Ia["dof_g"], Ia["dof_o"]
                gb, ob = Ib["dof_g"], Ib["dof_o"]

                # GLOBAL indices
                Gga, Goa = off1 + ga, off1 + oa
                Ggb, Gob = off1 + gb, off1 + ob

                # === penalty ===
                pen = eta_gamma / h_gamma
                for (I, sI) in [(Gga, +1.0), (Ggb, -1.0)]:
                    for (J, sJ) in [(Gga, +1.0), (Ggb, -1.0)]:
                        add_entry(I, J, pen * sI * sJ)

                # === New jump: -(1/m) {σ(u)}^{ab} (v_ga - v_gb) ===
                w = -1.0 / m
                for (Row, srow) in [(Gga, +1.0), (Ggb, -1.0)]:
                    # flux_a(u) = Da*(u_oa - u_ga)/ha
                    add_entry(Row, Goa, w * srow * ( Da/ha))
                    add_entry(Row, Gga, w * srow * (-Da/ha))
                    # -flux_b(u) = -Db*(u_ob - u_gb)/hb
                    add_entry(Row, Gob, w * srow * (-Db/hb))
                    add_entry(Row, Ggb, w * srow * ( Db/hb))

                # === New average: -(ε/m) {σ(v)}^{ab} (u_ga - u_gb) ===
                ws = -epsilon / m
                for (Col, scol) in [(Gga, +1.0), (Ggb, -1.0)]:
                    # flux_a(v) rows
                    add_entry(Goa, Col, ws * scol * ( Da/ha))
                    add_entry(Gga, Col, ws * scol * (-Da/ha))
                    # -flux_b(v) rows
                    add_entry(Gob, Col, ws * scol * (-Db/hb))
                    add_entry(Ggb, Col, ws * scol * ( Db/hb))

    # finalize PETSc assembly
    M.assemblyBegin()
    M.assemblyEnd()

def kirchhoff_residual_at_vertex(Lambda, V1, uh1d, vtx, D_area_expr):
  coords = Lambda.coordinates()
  dm = V1.dofmap()

  Lambda.init(0, 1)
  v2c = Lambda.topology()(0, 1)
  cells = list(v2c(vtx))

  res = 0.0
  fluxes = []
  for c in cells:
      cell = Cell(Lambda, c)
      vids = cell.entities(0)
      x0 = coords[vids[0]]
      x1 = coords[vids[1]]
      hK = float(np.linalg.norm(x1 - x0))
      x_mid = 0.5*(x0 + x1)
      Da = float(D_area_expr(Point(*x_mid)))

      cdofs = dm.cell_dofs(c)
      # local dof at junction vs other endpoint
      if vids[0] == vtx:
          dof_g, dof_o = int(cdofs[0]), int(cdofs[1])
      else:
          dof_g, dof_o = int(cdofs[1]), int(cdofs[0])

      ug = uh1d.vector()[dof_g]
      uo = uh1d.vector()[dof_o]
      F = Da * (uo - ug) / hK  # outgoing flux from junction into cell
      fluxes.append(F)
      res += F

  return res, fluxes

def advection_junction_terms_P1DG(
    A_petsc, V3, V1, Lambda, bif_vertices, D_area_expr,
    q1_value=1.0
):
    dm = V1.dofmap()
    coords = Lambda.coordinates()

    Lambda.init(0, 1)
    v2c = Lambda.topology()(0, 1)

    # Use the same safe offset as for diffusion:
    N = A_petsc.mat().getSize()[0]
    off1 = N - V1.dim()   # V1 block last

    M = A_petsc.mat()
    M.setOption(PETSc.Mat.Option.NEW_NONZERO_ALLOCATION_ERR, False)
    M.setOption(PETSc.Mat.Option.NEW_NONZERO_LOCATION_ERR, False)

    def add_entry(I, J, val):
        M.setValue(I, J, float(val), addv=True)

    for vtx in bif_vertices:
        cells = list(v2c(vtx))
        if len(cells) != 3:
            continue  # expecting a Y junction

        xg = coords[vtx]
        branches = []

        # 1) collect branch data
        for c in cells:
            cell = Cell(Lambda, c)
            vids = cell.entities(0)
            x0, x1 = coords[vids[0]], coords[vids[1]]

            if vids[0] == vtx:
                loc_g = 0
                other = x1
                x_mid = 0.5*(x0 + x1)
            else:
                loc_g = 1
                other = x0
                x_mid = 0.5*(x0 + x1)

            dof_g = int(dm.cell_dofs(c)[loc_g])
            A_here = float(D_area_expr(Point(*x_mid)))
            Q_here = A_here * float(q1_value)

            branches.append({
                "dof_g": dof_g,
                "Q": Q_here,
                "z_other": other[2],
            })

        # 2) classify: inflow = smallest z_other (parent), outflows = other two
        inflow = min(branches, key=lambda b: b["z_other"])
        outflows = [b for b in branches if b is not inflow]

        Gin = off1 + inflow["dof_g"]

        # 3) assemble: sum_out Q_out * u_in * (v_out - v_in)
        Qsum = 0.0
        for b in outflows:
            Gout = off1 + b["dof_g"]
            Q = b["Q"]
            Qsum += Q
            add_entry(Gout, Gin, +Q)   # v_out * u_in
        add_entry(Gin, Gin, -Qsum)     # -v_in * u_in

    M.assemblyBegin()
    M.assemblyEnd()

def build_cell_tangent_expression(Lambda, degree=0):
    coords = Lambda.coordinates()
    cell_mids = []
    cell_tans = []

    for c in range(Lambda.num_cells()):
        cell = Cell(Lambda, c)
        vids = cell.entities(0)
        x0, x1 = coords[vids[0]], coords[vids[1]]

        t = x1 - x0
        t /= np.linalg.norm(t)

        # === ORIENT tangent consistently (toward +z) ===
        # this is specific to this network need to come back and revise for general
        if t[2] < 0.0:
            t = -t

        cell_mids.append(0.5*(x0 + x1))
        cell_tans.append(t)

    cell_mids = np.array(cell_mids)
    cell_tans = np.array(cell_tans)
    kdt = cKDTree(cell_mids)

    class CellTangent(UserExpression):
        def eval(self, values, x):
            _, idx = kdt.query((x[0], x[1], x[2]))
            values[:] = cell_tans[idx]
        def value_shape(self): return (3,)

    return CellTangent(degree=degree)

def build_bfs_tangent_expression(Lambda, inlet_v, degree=0):
    coords = Lambda.coordinates()
    conn   = Lambda.cells()  # (v0,v1) per cell

    # === BFS distances on the vertex graph ===
    adj = {v: [] for v in range(Lambda.num_vertices())}
    for a, b in conn:
        a = int(a); b = int(b)
        adj[a].append(b); adj[b].append(a)

    dist = {int(inlet_v): 0}
    q = deque([int(inlet_v)])
    while q:
        v = q.popleft()
        for nb in adj[v]:
            if nb not in dist:
                dist[nb] = dist[v] + 1
                q.append(nb)

    # === per-cell downstream tangent (from smaller dist -> larger dist) ===
    cell_mids = []
    cell_tans = []

    for c, (a, b) in enumerate(conn):
        a = int(a); b = int(b)
        da = dist.get(a, None)
        db = dist.get(b, None)
        if da is None or db is None:

            up, dn = a, b
        else:
            up, dn = (a, b) if da <= db else (b, a)

        x_up = coords[up]
        x_dn = coords[dn]
        t = x_dn - x_up
        t /= np.linalg.norm(t)

        cell_mids.append(0.5*(x_up + x_dn))
        cell_tans.append(t)

    cell_mids = np.array(cell_mids)
    cell_tans = np.array(cell_tans)
    kdt = cKDTree(cell_mids)

    class BFSTangent(UserExpression):
        def eval(self, values, x):
            _, idx = kdt.query((x[0], x[1], x[2]))
            values[:] = cell_tans[idx]
        def value_shape(self):
            return (3,)

    return BFSTangent(degree=degree)

def outlet_bn_geometric(Lambda, endpoints, Lambda_boundaries, t_vec, q1_value=1.0):
    coords = Lambda.coordinates()
    Lambda.init(0, 1)
    v2e = Lambda.topology()(0, 1)
    e2v = Lambda.topology()(1, 0)

    for v in endpoints:
        if Lambda_boundaries.array()[v] != 2:
            continue  # only outlets

        # endpoint has exactly one incident edge
        e = int(v2e(v)[0])
        vids = e2v(e)

        # other vertex on that edge
        v_other = int(vids[0] if vids[1] == v else vids[1])

        x_end   = coords[v]
        x_other = coords[v_other]

        # outward direction from interior -> endpoint
        d = x_end - x_other
        d_hat = d / np.linalg.norm(d)

        # evaluate your tangent (UserExpression) at edge midpoint
        x_mid = 0.5 * (x_end + x_other)
        tmid = np.array(t_vec(Point(*x_mid)))  # this IS ok; it's your UserExpression
        tmid = tmid / np.linalg.norm(tmid)

        bn = float(q1_value) * float(np.dot(tmid, d_hat))
        print(f"outlet vertex {v}: other={v_other}, bn≈{bn:+.6e}, tmid={tmid}, dout={d_hat}")

def branch_cells_from_outlet(Lambda, bif_vtx, outlet_vtx):
    """Return ordered list of cell indices from bif_vtx -> outlet_vtx."""
    coords = Lambda.coordinates()
    Lambda.init(0, 1)
    v2c = Lambda.topology()(0, 1)

    # In 1D, outlet vertex touches exactly 1 cell
    current_cell = int(v2c(outlet_vtx)[0])
    cells = [current_cell]

    # Walk toward the bifurcation: each interior vertex has 2 incident cells
    while True:
        cell = Cell(Lambda, current_cell)
        vids = cell.entities(0)

        # pick the vertext closest to bifurcation
        # Step by finding which vertex of this cell has degree 2 (interior) until we reach bif_vtx.
        if bif_vtx in vids:
            break  # reached bifurcation cell

        # choose the vertex that is not the endpoint and keep moving
        # Candidate next vertices are vids[0], vids[1]; one of them leads to another cell.
        next_vertex = None
        for v in vids:
            if v == outlet_vtx:
                continue
            # if this vertex is not the outlet and has 2 incident cells, it is interior
            if len(v2c(v)) == 2:
                next_vertex = v
                break

        if next_vertex is None:
            # fallback: choose vertex that is not outlet_vtx
            next_vertex = vids[0] if vids[1] == outlet_vtx else vids[1]

        # Move to the other incident cell across next_vertex
        inc = list(v2c(next_vertex))
        next_cell = inc[0] if inc[1] == current_cell else inc[1]

        current_cell = int(next_cell)
        cells.append(current_cell)

    return list(reversed(cells))  # bif -> outlet order

def rel_diff(a, b):
    return abs(a-b) / max(1e-16, 0.5*(abs(a)+abs(b)))

def classify_branch_cells(Lambda, bif_vtx):
    coords = Lambda.coordinates()
    bif_x = coords[bif_vtx]              # numpy array shape (3,)

    left = []
    right = []

    for c in range(Lambda.num_cells()):
        cell = Cell(Lambda, c)
        mpP = cell.midpoint()            # dolfin Point
        mp  = np.array(mpP.array())      # numpy array (3,)

        d = mp - bif_x                   # numpy - numpy works

        # ignore parent branch (z <= 0 relative to bif)
        if d[2] <= 0.0:
            continue

        # classify by x sign
        if mp[0] > 0.0:
            right.append((np.linalg.norm(d), c))
        else:
            left.append((np.linalg.norm(d), c))

    left.sort()
    right.sort()

    return [c for _, c in left], [c for _, c in right]

class radius_function(UserExpression):
    def __init__(self, G, node_list, kdtree, **kwargs):
        self.G = G
        self.node_list = node_list
        self.kdtree = kdtree
        super().__init__(**kwargs)
    def eval(self, value, x):
        _, idx = self.kdtree.query((x[0], x[1], x[2]))
        node = self.node_list[idx]
        value[0] = float(self.G.nodes[node]['radius'])
    def value_shape(self):
        return ()
# Currently using this function due to how mesh was created
class edge_radius_function(UserExpression):
    def __init__(self, edge_midpoints, edge_radii, kdtree, **kwargs):
        super().__init__(**kwargs)
        self.edge_midpoints = edge_midpoints
        self.edge_radii = edge_radii
        self.kdtree = kdtree

    def eval(self, value, x):
        _, idx = self.kdtree.query((x[0], x[1], x[2]))
        value[0] = float(self.edge_radii[idx])

    def value_shape(self):
        return ()

/usr/local/lib/python3.12/dist-packages/block/__init__.py:15: UserWarning: The cbc.block repository has moved to https://github.com/blocknics/cbc.block
  warnings.warn('The cbc.block repository has moved to https://github.com/blocknics/cbc.block', UserWarning)


Missing HsMG for fract norm computing


In [ ]:
#@title dG Y bifurcation


# === Parameters ===
kappa     = Constant(0.1)
gamma_in  = 1.0
gamma_out = 1.0

mesh_vec = [2]

T    = 1.0

vtu3 = File("3d_Prediction_dGY_series.pvd") #1d prediction to be written
vtu1 = File("1d_Prediction_dGY_series.pvd") #3d prediction to be written

for i in mesh_vec:
  ncell   = 4*(2**i)
  G       = make_Y_bifurcation()
  mesh_3d = BoxMesh(Point(-0.5,-0.5,-0.5),Point(0.5,0.5,0.5),ncell,ncell,ncell)
  radius  = 0.05
  dt      = 0.1#/ncell
  inv_dt  = Constant(1.0/dt)
  tpts    = int(T/dt)

  # === Create Meshes ===
  G.make_mesh()
  Lambda, mf_Lambda = G.get_mesh()
  nodes, P_nodes, T_nodes = bif_tangents(G)

  Omega_boundaries = MeshFunction("size_t", mesh_3d, mesh_3d.topology().dim() - 1, 0)
  CompiledSubDomain("near(x[2], -0.5)").mark(Omega_boundaries, 1)
  CompiledSubDomain("near(x[2],  0.5)").mark(Omega_boundaries, 2)

  Lambda_boundaries = MeshFunction("size_t", Lambda, 0, 0)  # vertices
  coords = Lambda.coordinates()

  Lambda.init(0,1); v2e = Lambda.topology()(0,1)
  endpoints = [v for v in range(Lambda.num_vertices()) if v2e.size(v)==1]
  inlet_v = min(endpoints, key=lambda v: coords[v,2])
  t_vec = build_bfs_tangent_expression(Lambda, inlet_v, degree=0)

  Lambda_boundaries.array()[inlet_v] = 1
  for v in endpoints:
      if v != inlet_v:
          Lambda_boundaries.array()[v] = 2

  outlet_bn_geometric(Lambda, endpoints, Lambda_boundaries, t_vec, q1_value=1.0)


  # === Build edge-midpoint KDTree for radius mapping ===
  pos = nx.get_node_attributes(G, "pos")

  edge_midpoints = []
  edge_radii = []

  for (a, b) in G.edges():
      pa = np.array(pos[a])
      pb = np.array(pos[b])
      pmid = 0.5*(pa + pb)

      r_edge = float(G.nodes[b]["radius"])

      edge_midpoints.append(pmid)
      edge_radii.append(r_edge)

  edge_midpoints = np.array(edge_midpoints)
  edge_radii = np.array(edge_radii)
  edge_kdtree = cKDTree(edge_midpoints)

  radius_map_G = edge_radius_function(edge_midpoints, edge_radii, edge_kdtree, degree=1)

  # === Perimeter and Area dependent on position ===
  D_area       = np.pi * radius_map_G**2
  D_perimeter  = 2.0  * np.pi * radius_map_G
  cylinder = Circle(radius=radius_map_G, degree=10)

  # === Function Spaces ===
  V3 = FunctionSpace(mesh_3d, "DG", 1)
  V1 = FunctionSpace(Lambda, "DG", 1)
  W = [V3, V1]
  u3, u1 = map(TrialFunction, W)
  v3, v1 = map(TestFunction, W)


  # === Initial concentration ===
  cinlet = Constant(5.0) # Amount of drug administered
  tau = 0.10  # Duration of administration
  pulse = Constant(0.0) # needed for conditional to stop drug administration

  # === Bifurcations ===
  bif_info = build_bifurcation_pairs(G) # constructing network tree
  bif_vertices = bifurcation_vertices_from_graph(Lambda, bif_info)
  left_cells, right_cells = classify_branch_cells(Lambda, bif_vertices[0])
  outlet_vs = [v for v in endpoints if Lambda_boundaries.array()[v] == 2]

  # === Mesh Measures ===
  dxOmega = Measure("dx", domain=mesh_3d)
  dsOmega = Measure("ds", domain=mesh_3d, subdomain_data = Omega_boundaries)
  dSOmega = Measure("dS", domain=mesh_3d)
  dxLambda = Measure("dx", domain=Lambda)
  dsLambda = Measure("ds", domain=Lambda, subdomain_data=Lambda_boundaries)
  dSLambda = Measure("dS", domain=Lambda)
  n3d      = FacetNormal(mesh_3d)
  n1d      = FacetNormal(Lambda)
  h3d        = CellDiameter(mesh_3d)
  h1d        = CellDiameter(Lambda)
  alpha3d    = Constant(50.0)
  alpha1d    = Constant(50.0)
  alpha_bif  = Constant(50.0)
  alpha_boundary = Constant(50.0)
  epsilon  = Constant(1.0)
  perf1    = 1.0
  perf3    = 1.0
  g = Constant(0.0)

  # === Velocity terms ===
  q1     = Constant(1.0)
  q_t    = q1 * t_vec
  q3     = as_vector((0,0,q1))
  q3_pos = dot(q3,n3d('+'))

  # === Upwind ===
  q_norm_1d = dot(q_t('+'),n1d('+'))
  u1_up   = conditional(ge(q_norm_1d, 0.0), u1('-'), u1('+'))
  u3_up   = conditional(ge(q3_pos, 0.0), u3('-'), u3('+'))

  # === Averages ===
  u3_avg = Average(u3, Lambda, cylinder)
  v3_avg = Average(v3, Lambda, cylinder)


  # === Initialize time step ===
  uh3d_prev = interpolate(Expression('0.0',degree=1), V3)
  uh1d_prev = interpolate(Expression('0.0',degree=1), V1)

  vtu3 = File("3d_Prediction_dGY_series.pvd") #1d prediction to be written
  vtu1 = File("1d_Prediction_dGY_series.pvd") #3d prediction to be written


  # === Begin time-stepping ===
  t = 0.0
  for step in range(tpts):
    t += dt
    print(f"[INFO] Time step {step+1}/{tpts}, t = {t:.3}")
    pulse.assign(1.0 if t <= tau else 0.0) # injecting concentration only at the start of time period

    #cinlet.t = t #include if time dependent concentration

    # === Variational Forms ===
    a00 = inv_dt * (inner(u3, v3) * dxOmega) \
          + inner(grad(u3), grad(v3)) * dxOmega \
          + D_perimeter * kappa * inner(u3_avg, v3_avg) * dxLambda \
          - epsilon * dot(avg(grad(v3)),jump(u3,n3d))* dSOmega \
          - dot(jump(v3,n3d),avg(grad(u3))) * dSOmega \
          + (alpha3d/avg(h3d)) * dot(jump(v3,n3d),jump(u3,n3d)) * dSOmega \
          - inner(q3, grad(v3)) * u3 * dxOmega \
          + q3_pos * u3_up * jump(v3) * dSOmega \
          + dot(q3,n3d) * u3 * v3 * dsOmega(2)
          #- epsilon * dot(grad(v3),u3*n3d) * dsOmega - dot(v3*n3d,grad(u3)) * dsOmega \
          #+ (alpha_boundary/h3d)*v3*u3*dsOmega \
    a01 = -kappa * D_perimeter * inner(u1, v3_avg) * dxLambda
    a10 = -kappa * D_perimeter * inner(u3_avg, v1) * dxLambda
    a11 = D_area * inv_dt * u1 * v1 * dxLambda \
          + D_area * inner(grad(u1), grad(v1)) * dxLambda \
          - D_area * inner(avg(grad(u1)), jump(v1, n1d)) * dSLambda \
          - epsilon * D_area * inner(avg(grad(v1)), jump(u1, n1d)) * dSLambda \
          + (alpha1d/avg(h1d)) * jump(u1) * jump(v1) * dSLambda \
          + kappa * D_perimeter * u1 * v1 * dxLambda \
          - D_area * u1 * dot(q_t, grad(v1)) * dxLambda \
          + D_area*q_norm_1d*u1_up*jump(v1)*dSLambda \
          + D_area * dot(q_t,n1d) * u1 * v1 * dsLambda(2)


    f3 = Constant(0.0)
    f1 = Constant(0.0)

    L0  = inner(f3, v3) * dxOmega + inv_dt * inner(uh3d_prev, v3) * dxOmega \
        #+ (alpha_boundary/h3d)*v3*g*dsOmega - epsilon*dot(grad(v3), g*n3d)*dsOmega
    L1  = inner(f1, v1) * dxLambda + D_area * inv_dt * uh1d_prev * v1 * dxLambda \
        + pulse * D_area * q1 * cinlet * v1 * dsLambda(1) \



    a = [[a00, a01], [a10, a11]]
    L = [L0, L1]

   # bc3d = DirichletBC(V3, u3_exact, "on_boundary")
    W_bcs = [[], []]  # Boundaries now included in variational form for dG

    A, b = map(ii_assemble, (a, L))
    A, b = apply_bc(A, b, W_bcs)
    A, b = map(ii_convert, (A, b))

    # === Kirchhoff junction terms (diffusion, DG1) ===
    add_kirchhoff_junction_terms_P1DG(
        A, V3, V1, Lambda, bif_vertices,
        D_area_expr=D_area,
        eta_gamma=10.0,             # paper uses 10
        epsilon=1.0      # same epsilon as in variational form for SIPG
    )
    A.apply("insert")

    advection_junction_terms_P1DG(A, V3, V1, Lambda, bif_vertices, D_area_expr=D_area, q1_value=1.0)
    A.apply("insert")

    solver = PETScKrylovSolver()
    solver.set_operators(A, A)
    ksp = solver.ksp()

    opts = PETSc.Options()
    opts.setValue('ksp_type', 'gmres')
    opts.setValue('ksp_norm_type', 'unpreconditioned')
    opts.setValue('ksp_atol', 1E-14)
    opts.setValue('ksp_rtol', 1E-30)
    opts.setValue('ksp_monitor_true_residual', None)
    opts.setValue('pc_type', 'hypre')
    ksp.setFromOptions()

    x = b.copy()
    solver.solve(A,x,b)

    wh = ii_Function(W)
    wh.vector()[:] = x
    uh3d, uh1d = wh

    uh3d_prev.assign(uh3d)
    uh1d_prev.assign(uh1d)

    #avg_uh3d = Average(uh3d, Lambda, cylinder)
    #avg_u3d_exact = Average(u3_exact, Lambda, cylinder)


    # === Store Model Predictions ===
    vtkfile = File('3d_Prediction_dGY_series.pvd')
    uh3d.rename("u3","u3")
    vtu3 << (uh3d, t)

    vtkfile = File('1d_Prediction_dGY_series.pvd')
    uh1d.rename("u1","u1")
    vtu1 << (uh1d, t)


  # === Final time-step only ===
  File("3d_Prediction_final_dGY.pvd") << uh3d
  File("1d_Prediction_final_dGY.pvd") << uh1d

Calling FFC just-in-time (JIT) compiler, this may take some time.


Level 25:FFC:Calling FFC just-in-time (JIT) compiler, this may take some time.
INFO:FFC:Compiling element ffc_element_86765630720c088a2ad90ee684d3948b5df76d5b

INFO:FFC:Compiler stage 1: Analyzing element(s)
INFO:FFC:--------------------------------------
INFO:FFC:  
INFO:FFC:Compiler stage 1 finished in 0.00240088 seconds.

INFO:FFC:Compiler stage 2: Computing intermediate representation
INFO:FFC:-------------------------------------------------------
INFO:FFC:  Computing representation of 1 elements
DEBUG:FFC:  Reusing element from cache
DEBUG:FFC:  Reusing element from cache
DEBUG:FFC:  Reusing element from cache
DEBUG:FFC:  Reusing element from cache
DEBUG:FFC:  Reusing element from cache
DEBUG:FFC:  Reusing element from cache
DEBUG:FFC:  Reusing element from cache
DEBUG:FFC:  Reusing element from cache
INFO:FFC:  Computing representation of 1 dofmaps
DEBUG:FFC:  Reusing element from cache
INFO:FFC:  Computing representation of 0 coordinate mappings
INFO:FFC:  Computing representat

Calling FFC just-in-time (JIT) compiler, this may take some time.


Level 25:FFC:Calling FFC just-in-time (JIT) compiler, this may take some time.
INFO:FFC:Compiling element ffc_element_22bb736b850c370ac1741e1a80b6653eae2ad213

INFO:FFC:Compiler stage 1: Analyzing element(s)
INFO:FFC:--------------------------------------
INFO:FFC:  
INFO:FFC:Compiler stage 1 finished in 0.00375605 seconds.

INFO:FFC:Compiler stage 2: Computing intermediate representation
INFO:FFC:-------------------------------------------------------
INFO:FFC:  Computing representation of 1 elements
DEBUG:FFC:  Reusing element from cache
DEBUG:FFC:  Reusing element from cache
DEBUG:FFC:  Reusing element from cache
INFO:FFC:  Computing representation of 1 dofmaps
DEBUG:FFC:  Reusing element from cache
INFO:FFC:  Computing representation of 0 coordinate mappings
INFO:FFC:  Computing representation of integrals
INFO:FFC:  Computing representation of forms
INFO:FFC:  
INFO:FFC:Compiler stage 2 finished in 0.0154774 seconds.

INFO:FFC:Compiler stage 3: Optimizing intermediate representati

outlet vertex 6: other=16, bn≈+1.000000e+00, tmid=[0.57357644 0.         0.81915204], dout=[0.57357644 0.         0.81915204]
outlet vertex 9: other=18, bn≈+1.000000e+00, tmid=[-0.57357644  0.          0.81915204], dout=[-0.57357644  0.          0.81915204]
Calling FFC just-in-time (JIT) compiler, this may take some time.


...

Calling FFC just-in-time (JIT) compiler, this may take some time.


Level 25:FFC:Calling FFC just-in-time (JIT) compiler, this may take some time.
INFO:FFC:Compiling form ffc_form_fd3bf13f876ac7c626762a280d843f5dba094dd0

INFO:FFC:Compiler stage 1: Analyzing form(s)
INFO:FFC:-----------------------------------
DEBUG:FFC:  Preprocessing form using 'uflacs' representation family.
INFO:UFL_LEGACY:Adjusting missing element cell to tetrahedron.
INFO:UFL_LEGACY:Adjusting missing element cell to tetrahedron.
INFO:FFC:  
INFO:FFC:  Geometric dimension:       3
  Number of cell subdomains: 0
  Rank:                      1
  Arguments:                 '(v_0)'
  Number of coefficients:    3
  Coefficients:              '[f_29, f_80, f_87]'
  Unique elements:           'DG1(?,?), R0(?,?), Vector<3 x CG1(?,?)>'
  Unique sub elements:       'DG1(?,?), R0(?,?), Vector<3 x CG1(?,?)>, CG1(?,?)'
  
INFO:FFC:  representation:    auto --> uflacs
INFO:FFC:  quadrature_rule:   auto --> default
INFO:FFC:  quadrature_degree: auto --> 2
INFO:FFC:  quadrature_degree: 2
INFO:FFC

Calling FFC just-in-time (JIT) compiler, this may take some time.


Level 25:FFC:Calling FFC just-in-time (JIT) compiler, this may take some time.
INFO:FFC:Compiling form ffc_form_ef657ee7c52f84a44a70760ac66f9e3496b4d206

INFO:FFC:Compiler stage 1: Analyzing form(s)
INFO:FFC:-----------------------------------
DEBUG:FFC:  Preprocessing form using 'uflacs' representation family.
INFO:UFL_LEGACY:Adjusting missing element cell to interval3D.
INFO:UFL_LEGACY:Adjusting missing element cell to interval3D.
INFO:UFL_LEGACY:Adjusting missing element cell to interval3D.
INFO:UFL_LEGACY:Adjusting missing element cell to interval3D.
INFO:UFL_LEGACY:Adjusting missing element cell to interval3D.
INFO:UFL_LEGACY:Adjusting missing element cell to interval3D.
INFO:FFC:  
INFO:FFC:  Geometric dimension:                 3
  Number of cell subdomains:           0
  Number of exterior_facet subdomains: 2
  Rank:                                1
  Arguments:                           '(v_0)'
  Number of coefficients:              7
  Coefficients:                        '[f

  0 KSP unpreconditioned resid norm 3.926990816987e-02 true resid norm 3.926990816987e-02 ||r(i)||/||b|| 1.000000000000e+00
  1 KSP unpreconditioned resid norm 8.125883662416e-04 true resid norm 8.125883662233e-04 ||r(i)||/||b|| 2.069239282934e-02
  2 KSP unpreconditioned resid norm 3.520376951701e-04 true resid norm 3.520376951698e-04 ||r(i)||/||b|| 8.964566294551e-03
  3 KSP unpreconditioned resid norm 1.598430122181e-04 true resid norm 1.598430122159e-04 ||r(i)||/||b|| 4.070368882058e-03
  4 KSP unpreconditioned resid norm 9.728388494562e-05 true resid norm 9.728388494472e-05 ||r(i)||/||b|| 2.477313787542e-03
  5 KSP unpreconditioned resid norm 4.780434240970e-05 true resid norm 4.780434241418e-05 ||r(i)||/||b|| 1.217327583436e-03
  6 KSP unpreconditioned resid norm 2.811350829273e-05 true resid norm 2.811350828993e-05 ||r(i)||/||b|| 7.159046099195e-04
  7 KSP unpreconditioned resid norm 1.614883408788e-05 true resid norm 1.614883410272e-05 ||r(i)||/||b|| 4.112266836191e-04
  8 KSP 